In [7]:
import os 
from langchain_community.tools.tavily_search import TavilySearchResults
from dotenv import load_dotenv
load_dotenv()
from langchain_groq import ChatGroq
from langchain_anthropic import ChatAnthropic
import time
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
import json
os.getenv("GROQ_API_KEY")
os.getenv("TAVILY_API_KEY")
os.getenv("ANTHROPIC_API_KEY")
groq_llm=ChatGroq(model="llama-3.1-70b-versatile")

search = TavilySearchResults()

In [8]:
groq_llm.invoke(" kendrick lamar and its probably a minor song")

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"


AIMessage(content="Kendrick Lamar has a vast discography with many great songs.  Since you mentioned it's probably a minor song, I'll need a bit more information to help you identify the song. Can you provide any details or clues about the song, such as:\n\n* The album it's from (e.g. Good Kid, M.A.A.D City, To Pimp a Butterfly, etc.)\n* A few lyrics from the song\n* The tone or style of the song (e.g. upbeat, slow, storytelling, etc.)\n* When you heard the song (approximate year or era)\n\nAny information you can provide will help me narrow down the options and try to identify the song you're thinking of.", response_metadata={'token_usage': {'completion_tokens': 145, 'prompt_tokens': 45, 'total_tokens': 190, 'completion_time': 0.58, 'prompt_time': 0.010816842, 'queue_time': 0.098838006, 'total_time': 0.590816842}, 'model_name': 'llama-3.1-70b-versatile', 'system_fingerprint': 'fp_9260b4bb2e', 'finish_reason': 'stop', 'logprobs': None}, id='run-042a63c4-23ac-45f1-9354-289b7c108c02-0')

In [9]:
import openpyxl
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from email.mime.text import MIMEText
import smtplib
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)



def read_excel_data(file_path, start_row):
    workbook = openpyxl.load_workbook(file_path)
    sheet = workbook.active
    data = []
    for row in sheet.iter_rows(min_row=start_row, values_only=True):
        if row[2] != "email not found":  # Assuming email is in the 3rd column (index 2)
            data.append({
                "meta_description": row[3],
                "username": row[0],
                "page_name": row[1],
                "email": row[2]
            })
    return data

def generate_email_body(page_name: str, meta_description: str):
    try:
        email_body_system_prompt = """

        Write a personalized email body offering AI automation services based on the provided data. 

Follow these guidelines:
    
**Strictly Don't Use Any Emojis and Don't Write any subject line or heading. Refer to the recipient by their first name only.**
1. Limit to 3 short paragraphs, each 1-2 lines long. Do not exceed this length.
2. Paragraph 1: Start with a personalized observation about the recipient's business based on the provided information. Then, introduce a specific example of how AI voice agents can help their business, including capturing lead information.
3. Paragraph 2: List 1-2 specific tasks AI agents can handle in their industry, relating them to the recipient's business context. Include a concrete benefit with numbers.
4. Final paragraph: End with a clear call-to-action offering a call or custom demo, and include the website link: https://optimoforge-ai.odoo.com/ai-agents#Image-Gallery.
5. Keep the tone professional yet conversational.
6. Ensure each paragraph serves a distinct purpose within the limited space.

Sample Example:
Hi [First Name],

I was impressed by [Company Name]'s innovative approach to [specific aspect of their business]. Our AI voice agents could enhance your [specific business area] by efficiently capturing lead information, ensuring no opportunity slips through the cracks.

For [industry-specific task], our AI agents can [specific action], potentially increasing efficiency by 30%. They can also [another relevant task], which has helped similar businesses boost their conversion rates by 25%.

I'd love to show you how this could work for [Company Name]. Are you free for a quick chat next week? Check out our AI agents in action: https://optimoforge-ai.odoo.com/ai-agents#Image-Gallery

Best regards,
Abdullah, OptimoForge AI
        """

        email_body_prompt = """
        Client Name: {page_name}
        Business intro: {meta_description}

        Craft a personalized cold email pitch for OptimoForge AI services based on the given information and following the guidelines.
        """

        email_body_prompt_template = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(email_body_system_prompt),
            HumanMessagePromptTemplate.from_template(email_body_prompt)
        ])

        formatted_email_body_prompt = email_body_prompt_template.format_messages(
            page_name=page_name,
            meta_description=meta_description
        )

        email_body_response = groq_llm.invoke(formatted_email_body_prompt)

        return email_body_response.content
    except Exception as e:
        logger.error(f"Error in generating email body: {str(e)}")
        return None

def generate_subject_line(email_body: str, business_info: str):
    try:
        subject_line_system_prompt = """

        You are an expert in selecting attention-grabbing email subject lines. Your task is to choose the most appropriate subject line from a given list and personalize it for the target client or company.

Guidelines:
- Do not use emojis
- Provide only the selected subject line, without any explanations
- Replace the company name in the chosen line with the target client's first name

Choose one line from the following options and personalize it by replacing Client's Name with the client's first name mentioned in the email body that will be provided:

- [Client's Name], Let's talk about working smarter, not harder.
Output: The selected subject line with the client's first name replacing the original company name.
        """

        subject_line_prompt = """
        Email Body:
        {email_body}

        Business Info:
        {business_info}

        Create a catchy subject line for the email based on the given email body and business info.

        """

        subject_line_prompt_template = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(subject_line_system_prompt),
            HumanMessagePromptTemplate.from_template(subject_line_prompt)
        ])

        formatted_subject_line_prompt = subject_line_prompt_template.format_messages(
            email_body=email_body,
            business_info=business_info
        )

        subject_line_response = groq_llm.invoke(formatted_subject_line_prompt)

        return subject_line_response.content
    except Exception as e:
        logger.error(f"Error in generating subject line: {str(e)}")
        return None
    

In [10]:
from pydantic import BaseModel, Field
from langchain.output_parsers import PydanticOutputParser
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

class CheckOutput(BaseModel):
    """Used to check if the output is relevant to our services"""
    is_relevant: str = Field(description="Check if the output is relevant to our AI automation services and follows the given guidelines. Give a one word answer either 'Yes' OR 'No' nothing else is accepted")

output_checker_parser = PydanticOutputParser(pydantic_object=CheckOutput)

def check_output_relevance(email_body: str, subject_line: str):
    try:
        checker_system_prompt = """
        **Guidelines for email body:
1. Should be 3 short paragraphs, each 1-2 lines long.
2. Must include specific examples of how AI voice agents can help in their business, including capturing lead information.
3. Should list 1-2 specific tasks AI agents can handle in their industry.
4. Must include concrete benefits with numbers.
5. Should end with a clear call-to-action offering a call and AI Voice Agent demo link https://optimoforge-ai.odoo.com/ai-agents#Image-Gallery.
6. Tone should be professional yet conversational.
7. No emojis should be used.
8. Refer to the recipient by their first name only.

**Guidelines for subject line:
1. Should be attention-grabbing and relevant to AI automation services.
2. Must not use emojis.
3. Should be personalized with the client's first name.

Only give a binary score of either 'Yes' if relevant and following guidelines, or 'No' if not.

        """

        checker_human_prompt = """
        Subject Line: {subject_line}
        
        Email Body:
        {email_body}

        Are both the subject line and email body relevant to our AI automation services and do they follow the guidelines?

        {format_instructions}
        """

        checker_prompt_template = ChatPromptTemplate.from_messages([
            SystemMessagePromptTemplate.from_template(checker_system_prompt),
            HumanMessagePromptTemplate.from_template(checker_human_prompt)
        ])

        formatted_checker_prompt = checker_prompt_template.format_messages(
            subject_line=subject_line,
            email_body=email_body,
            format_instructions=output_checker_parser.get_format_instructions()
        )

        checker_response = groq_llm.invoke(formatted_checker_prompt)

        if isinstance(checker_response, dict):
            is_relevant = checker_response.get('is_relevant', 'Unknown')
        elif isinstance(checker_response, CheckOutput):
            is_relevant = checker_response.is_relevant
        elif hasattr(checker_response, 'content'):
            # Handle AIMessage object
            content = checker_response.content.lower()
            is_relevant = 'Yes' if 'yes' in content else 'No'
        else:
            logger.error(f"Unexpected result type: {type(checker_response)}")
            return 'No'

        logger.info(f"Content Relevance: {is_relevant}")
        return is_relevant

    except Exception as e:
        logger.error(f"Error in checking output relevance: {str(e)}")
        return 'No'

In [11]:

from email.utils import formataddr

def send_email(smtp_server, smtp_port, email_sender, app_password, email_recipient, subject_line, email_body):
    try:
        # Prepare the email
        message = MIMEText(email_body)
        message['From'] = formataddr(("OptimoForge AI", email_sender))
        message['to'] = email_recipient
        message['subject'] = subject_line

        # Set up the SMTP server
        with smtplib.SMTP(smtp_server, smtp_port) as server:
            server.starttls()  # Start TLS encryption
            server.login(email_sender, app_password)  # Log in with your email and app password
            server.sendmail(email_sender, email_recipient, message.as_string())

        logger.info(f"Email sent to {email_recipient}")
    except Exception as e:
        logger.error(f"An error occurred while sending email to {email_recipient}: {e}")
        return None
def log_irrelevant_data(data, output_file='irrelevant_data.xlsx'):
    try:
        wb = openpyxl.Workbook()
        ws = wb.active
        # Use all fields from the original data, plus generated content
        ws.append(["username", "page_name", "email", "meta_description", "generated_email", "generated_subject"])
        ws.append([
            data['username'],
            data['page_name'],
            data['email'],
            data['meta_description'],
            data['generated_email'],
            data['generated_subject']
        ])
        wb.save(output_file)
        logger.info(f"Irrelevant data logged to {output_file}")
    except Exception as e:
        logger.error(f"Error logging irrelevant data: {str(e)}")


In [13]:

def main():
    smtp_server = "smtp.gmail.com"
    smtp_port = 587
    email_sender = "optimoforgeai@gmail.com"
    app_password = "xyrm ydbm zcyy usms"  

    excel_file_path = 'C:/Users/abdul/OneDrive/Documents/Leads.xlsx'
    start_row = int(input("Enter the starting row number: "))
    data = read_excel_data(excel_file_path, start_row)

    for row in data:
        email_body = generate_email_body(row['page_name'], row['meta_description'])
        if email_body:
            subject_line = generate_subject_line(email_body, f"{row['page_name']} - {row['meta_description']}")
            
            if subject_line:
                relevance_check = check_output_relevance(email_body, subject_line)
                if relevance_check.lower() == 'yes':
                    try:
                        send_email(smtp_server, smtp_port, email_sender, app_password, row['email'], subject_line, email_body)
                        logger.info(f"Email sent successfully to {row['email']}")
                    except Exception as e:
                        logger.error(f"Error sending email to {row['email']}: {e}")
                else:
                    logger.warning(f"Generated content for {row['email']} was deemed irrelevant. Logging for review.")
                    row['generated_email'] = email_body
                    row['generated_subject'] = subject_line
                    log_irrelevant_data(row)
            else:
                logger.error(f"Failed to generate subject line for {row['email']}")
        else:
            logger.error(f"Failed to generate email body for {row['email']}")

    logger.info("Email campaign completed!")

if __name__ == "__main__":
    main()

INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Content Relevance: Yes
INFO:__main__:Email sent to ricor@orhp.com
INFO:__main__:Email sent successfully to ricor@orhp.com
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:__main__:Content Relevance: Yes
INFO:__main__:Email sent to ricardo@mbi-usa.com
INFO:__main__:Email sent successfully to ricardo@mbi-usa.com
INFO:httpx:HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.groq

KeyboardInterrupt: 